In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import numpy as np

In [2]:
import ddddocr
from PIL import Image, ImageFilter

class MyGaussianBlur(ImageFilter.Filter):
    name = "GaussianBlur"
    
    def __init__(self, radius=2, bounds=None):
        self.radius = radius
        self.bounds = bounds
    def filter(self, image):
        if self.bounds:
            clips = image.crop(self.bounds).gaussian_blur(self.radius)
            image.paste(clips, self.bounds)
            return image
        else:
            return image.gaussian_blur(self.radius)

In [3]:
def ease_out_bounce(x):
    n1 = 7.5625
    d1 = 2.75
    if x < 1 / d1:
        return n1 * x * x
    elif x < 2 / d1:
        x -= 1.5 / d1
        return n1 * x * x + 0.75
    elif x < 2.5 / d1:
        x -= 2.25 / d1
        return n1 * x * x + 0.9375
    else:
        x -= 2.625 / d1
        return n1 * x * x + 0.984375


def ease_in_out_bounce(x):
    return ( 1 - ease_out_bounce( 1 - 2 * x ) ) / 2 if x < 0.5 else ( 1 + ease_out_bounce( 2 * x - 1 ) ) / 2


def get_tracks(distance, seconds, ease_func):
    """
    可以根据滑块的偏移，需要的时间（相对时间，并不是准确时间），以及要采用的缓动函数生成拖动轨迹。
    
    :param distance: 位移距离
    :param seconds: 时间秒
    :par
    am ease_func: 调用的缓慢函数
    :return:
    """
    tracks = [0]
    offsets = [0]
    for t in np.arange(0.0, seconds, 0.1):
        ease = globals()[ease_func]
        offset = round(ease(t / seconds) * distance)
        tracks.append(offset - offsets[-1])
        offsets.append(offset)
    return offsets, tracks

In [4]:
driver = webdriver.Chrome()
driver.get('https://www.tianyancha.com/')
time.sleep(2)

driver.find_element(By.CLASS_NAME, 'treasure_nav-link__7ErdH').click()
driver.find_element(By.XPATH, '//*[@id="J_Modal_Container"]/div/div/div[2]/div/div[2]/div/div/div[2]').click()
driver.find_element(By.CLASS_NAME, 'title-password').click()
time.sleep(2)

name = driver.find_element(By.XPATH, '//*[@id="J_Modal_Container"]/div/div/div[2]/div/div[2]/div/div/div[3]/div[2]/div[1]/input')
name.send_keys('18218869544')
time.sleep(2)
passwd = driver.find_element(By.XPATH, '//*[@id="J_Modal_Container"]/div/div/div[2]/div/div[2]/div/div/div[3]/div[2]/div[2]/input')
passwd.send_keys('YMH1308458539')
time.sleep(2)
check = driver.find_element(By.XPATH, '//*[@id="J_Modal_Container"]/div/div/div[2]/div/div[2]/div/div/div[3]/div[3]/input')
check.click()
time.sleep(1)
login = driver.find_element(By.XPATH, '//*[@id="J_Modal_Container"]/div/div/div[2]/div/div[2]/div/div/div[3]/div[2]/button/span')
login.click()
time.sleep(2)

while True:
    driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[2]/div[1]/div[2]/div[1]/a[2]/div[1]').screenshot('a.png')
    driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[2]/div[2]/div[2]').click()
    time.sleep(3)
    driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[2]/div[1]/div[2]/div[1]/a[2]/div[1]').screenshot('b.png')
    driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[2]/div[1]/div[2]/div[1]/a[1]/div[2]').screenshot('c.png')

    ocr = ddddocr.DdddOcr(det=False, ocr=False, show_ad=False)
    with open('b.png', 'rb') as f:
        background = f.read()
    with open('c.png', 'rb') as f:
        target = f.read()

    res_a = ocr.slide_match(target, background, simple_target=True)
    bounds = res_a['target']
    print(res_a)

    img = Image.open('b.png')
    img = img.filter(MyGaussianBlur(radius=29, bounds=bounds))
    img.save('b_after.png')

    with open('b_after.png', 'rb') as f:
        background = f.read()
    with open('c.png', 'rb') as f:
        target = f.read()

    res = ocr.slide_match(target, background, simple_target=True)
    bounds = res['target']
    print(res)

    img = Image.open('b.png')
    img = img.filter(MyGaussianBlur(radius=29, bounds=bounds))
    img.save('b_before.png')

    init_x = res['target'][0] - res_a['target'][2] + 10
    distance = res_a['target'][2] - 10
    # distance = res['target'][0] - res_a['target'][0] + 2  # 滑块移动的距离，就是上面识别到的第一个差异点的w值
    # dis_a, dis_b = distance // 2, distance // 2
    # track_a = create_track(dis_a)
    # track_b = create_track(dis_b)
    # _, track_a = get_tracks(dis_a, 1, 'ease_in_out_bounce')
    # _, track_b = get_tracks(dis_b, 2, 'ease_in_out_bounce')
    _, track = get_tracks(distance, 2, 'ease_in_out_bounce')

    from selenium.webdriver import ActionChains

    # 移动滑块
    actions = ActionChains(driver)  # 创建动作链
    slider = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[2]/div[2]/div[2]')  # 运用Xpath方法获取滑块元素
    actions.click_and_hold(slider).perform()  # 点击滑块，并保持点击动作
    # for x in track_a:
    #     actions.move_by_offset(xoffset=x, yoffset=0).perform()  # 拖动滑块
    # for x in track_b:
    #     actions.move_by_offset(xoffset=x, yoffset=0).perform()  # 拖动滑块
    actions.move_by_offset(xoffset=init_x, yoffset=0).perform()
    for x in track:
        actions.move_by_offset(xoffset=x, yoffset=0).perform()  # 拖动滑块
    time.sleep(1)
    actions.release().perform()  # 松开滑块，验证完成
    time.sleep(6)

{'target_y': 0, 'target': [0, 0, 53, 52]}
{'target_y': 0, 'target': [135, 0, 188, 52]}


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[4]/div[2]/div[2]/div[1]/div[2]/div[1]/a[2]/div[1]"}
  (Session info: chrome=115.0.5790.110); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00007FF6A5154A62+57106]
	(No symbol) [0x00007FF6A50CCF52]
	(No symbol) [0x00007FF6A4F9E2CB]
	(No symbol) [0x00007FF6A4FD786E]
	(No symbol) [0x00007FF6A4FD795C]
	(No symbol) [0x00007FF6A5010477]
	(No symbol) [0x00007FF6A4FF69FF]
	(No symbol) [0x00007FF6A500E522]
	(No symbol) [0x00007FF6A4FF6793]
	(No symbol) [0x00007FF6A4FCCE81]
	(No symbol) [0x00007FF6A4FCE064]
	GetHandleVerifier [0x00007FF6A5404222+2873042]
	GetHandleVerifier [0x00007FF6A5456590+3209792]
	GetHandleVerifier [0x00007FF6A544F3AF+3180639]
	GetHandleVerifier [0x00007FF6A51E5F25+652245]
	(No symbol) [0x00007FF6A50D8618]
	(No sy

In [31]:
import ddddocr
from PIL import Image, ImageFilter

class MyGaussianBlur(ImageFilter.Filter):
    name = "GaussianBlur"
    
    def __init__(self, radius=2, bounds=None):
        self.radius = radius
        self.bounds = bounds
    def filter(self, image):
        if self.bounds:
            clips = image.crop(self.bounds).gaussian_blur(self.radius)
            image.paste(clips, self.bounds)
            return image
        else:
            return image.gaussian_blur(self.radius)

ocr = ddddocr.DdddOcr(det=False, ocr=False, show_ad=False)
with open('b.png', 'rb') as f:
    background = f.read()
with open('c.png', 'rb') as f:
    target = f.read()

res_a = ocr.slide_match(target, background, simple_target=True)
bounds = res_a['target']
print(res_a)

{'target_y': 0, 'target': [0, 63, 55, 114]}


In [32]:
img = Image.open('b.png')
img = img.filter(MyGaussianBlur(radius=29, bounds=bounds))
img.save('b_after.png')

In [33]:
with open('b_after.png', 'rb') as f:
    background = f.read()
with open('c.png', 'rb') as f:
    target = f.read()

res = ocr.slide_match(target, background, simple_target=True)
bounds = res['target']
print(res)

{'target_y': 0, 'target': [131, 63, 186, 114]}


In [34]:
# 移动轨迹计算
distance = res['target'][0] - res_a['target'][0]  # 滑块移动的距离，就是上面识别到的第一个差异点的w值
track = []  # 移动轨迹，里面保留的是每次移动的距离
current = 0  # 当前位移
mid = distance * 3 / 4  # 减速阈值，前面4/5做匀加速移动，后面1/5做匀加速移动
t = 0.2  # 计算间隔，每次移动两秒
v = 0  # 初速度
while current < distance:  # 当移动的距离超过，目标距离后，就停止循环
    if current < mid:
        a = 3  # 前面匀加速的加速度为正2
    else:
        a = -2  # 后面匀减速的加速度为负3
    v0 = v  # 初速度 v0
    v = v0 + a * t  # 当前速度 v = v0 + at
    move = v0 * t + 1 / 2 * a * t * t  # 移动距离 x = v0t + 1/2 * a * t^2
    current = current + move  # 总位移
    track.append(round(move))  # 每次移动的距离
track[-1]=distance-sum(track[0:-1])  # 上面计算的总距离，会超过目标距离，所以最后一个移动距离需要修正一下

In [39]:
from selenium.webdriver import ActionChains

# 移动滑块
actions = ActionChains(driver)  # 创建动作链
slider = driver.find_element(By.XPATH, '/html/body/div[4]/div[2]/div[2]/div[2]/div[2]')  # 运用Xpath方法获取滑块元素
actions.click_and_hold(slider).perform()  # 点击滑块，并保持点击动作
for x in track:
    actions.move_by_offset(xoffset=x, yoffset=0).perform()  # 拖动滑块
time.sleep(1)
actions.release().perform()  # 松开滑块，验证完成

NameError: name 'browser' is not defined

In [66]:
def ease_out_bounce(x):
    n1 = 7.5625
    d1 = 2.75
    if x < 1 / d1:
        return n1 * x * x
    elif x < 2 / d1:
        x -= 1.5 / d1
        return n1 * x * x + 0.75
    elif x < 2.5 / d1:
        x -= 2.25 / d1
        return n1 * x * x + 0.9375
    else:
        x -= 2.625 / d1
        return n1 * x * x + 0.984375


def ease_in_out_bounce(x):
    return ( 1 - ease_out_bounce( 1 - 2 * x ) ) / 2 if x < 0.5 else ( 1 + ease_out_bounce( 2 * x - 1 ) ) / 2


def get_tracks(distance, seconds, ease_func):
    """
    可以根据滑块的偏移，需要的时间（相对时间，并不是准确时间），以及要采用的缓动函数生成拖动轨迹。
    
    :param distance: 位移距离
    :param seconds: 时间秒
    :par
    am ease_func: 调用的缓慢函数
    :return:
    """
    tracks = [0]
    offsets = [0]
    for t in np.arange(0.0, seconds, 0.1):
        ease = globals()[ease_func]
        offset = round(ease(t / seconds) * distance)
        tracks.append(offset - offsets[-1])
        offsets.append(offset)
    return offsets, tracks

In [71]:
def create_track(distance):
    track = []  # 移动轨迹，里面保留的是每次移动的距离
    current = 0  # 当前位移
    mid = distance * 3 / 4  # 减速阈值，前面4/5做匀加速移动，后面1/5做匀加速移动
    t = 0.5  # 计算间隔，每次移动两秒
    v = 0  # 初速度
    while current < distance:  # 当移动的距离超过，目标距离后，就停止循环
        if current < mid:
            a = 6  # 前面匀加速的加速度为正2
        else:
            a = -7  # 后面匀减速的加速度为负3
        v0 = v  # 初速度 v0
        v = v0 + a * t  # 当前速度 v = v0 + at
        move = v0 * t + 1 / 2 * a * t * t  # 移动距离 x = v0t + 1/2 * a * t^2
        current = current + move  # 总位移
        track.append(round(move))  # 每次移动的距离
    track[-1]=distance-sum(track[0:-1])  # 上面计算的总距离，会超过目标距离，所以最后一个移动距离需要修正一下
    return track

In [20]:
from sklearn.metrics import confusion_matrix

y = [0, 0, 1, 0, 1, 1, 1]
y_t = [0, 0, 1, 1, 0, 1, 1]
print(confusion_matrix(y, y_t).ravel())
a, b, c, d = confusion_matrix(y, y_t).ravel()

[2 1 1 3]


In [23]:
y = [0, 0, 0, 0, 0, 0, 0]
y_t = [0, 0, 0, 0, 0, 1, 0]
a, b, c, d = confusion_matrix(y, y_t).ravel()

In [24]:
confusion_matrix(y, y_t).ravel()

array([6, 1, 0, 0], dtype=int64)